In [1]:
#!/usr/bin/env python2

from __future__ import division
from __future__ import with_statement
from __future__ import print_function

import numpy
import deepmodels
import json
import time
import argparse
import os.path
import subprocess
import imageutils
import utils
import deepmodels_torch


In [2]:
with open('datasets/lfw/lfw_binary_attributes.json') as f: lfw=json.load(f)
with open('datasets/lfw/filelist.txt','r') as f: lfw_filelist=['images/'+x.strip() for x in f.readlines()]

In [3]:
type(lfw_filelist)

list

In [4]:
backend = 'torch'
device_id = 0
K = 100
scaling = 'beta'
itera = 500
postprocess=set('color'.split(','))

In [5]:
def make_manifolds(a,s=[],t=[],N=10,X=None,visualize=False):
    '''
    a is the target attribute, s are exclusive attributes for the source,
    t are exclusive attributes for the target.
    '''
    S={k:set(v) for k,v in lfw['attribute_members'].items()}
    T=lfw['attribute_gender_members']
    G=set(T[lfw['attribute_gender'][a]])
    if X is None:
        # test has correct gender, all of the source attributes and none of the target attributes
        X=[i for i in range(len(lfw_filelist)) if i in G and i not in S[a] and not any(i in S[b] for b in t) and all(i in S[b] for b in s)]
        random.seed(123)
        random.shuffle(X)
    else:
        X=[lfw_filelist.index(x) for x in X]

    def distfn(y,z):
        fy=[True if y in S[b] else False for b in sorted(S.keys())]
        fz=[True if z in S[b] else False for b in sorted(S.keys())]
        return sum(0 if yy==zz else 1 for yy,zz in zip(fy,fz))
    # source has correct gender, all of the source attributes and none of the target attributes
    # ranked by attribute distance to test image
    P=[i for i in range(len(lfw_filelist)) if i in G and i not in S[a] and not any(i in S[b] for b in t) and all(i in S[b] for b in s)]
    P=[sorted([j for j in P if j!=X[i]],key=lambda k: distfn(X[i],k)) for i in range(N)]
    # target has correct gender, none of the source attributes and all of the target attributes
    Q=[i for i in range(len(lfw_filelist)) if i in G and i in S[a] and not any(i in S[b] for b in s) and all(i in S[b] for b in t)]
    Q=[sorted([j for j in Q if j!=X[i] and j not in P[i]],key=lambda k: distfn(X[i],k)) for i in range(N)]

    return [lfw_filelist[x] for x in X],[[lfw_filelist[x] for x in y] for y in P],[[lfw_filelist[x] for x in y] for y in Q]

In [6]:
minimum_resolution=200
if backend=='torch':
    model=deepmodels_torch.vgg19g_torch(device_id=device_id)
elif backend=='caffe+scipy':
    model=deepmodels.vgg19g(device_id=device_id)
else:
    raise ValueError('Unknown backend')

In [7]:
if not os.path.exists('images/lfw_aegan'):
    url='https://www.dropbox.com/s/isz4ske2kheuwgr/lfw_aegan.tar.gz?dl=1'
    subprocess.check_call(['wget',url,'-O','lfw_aegan.tar.gz'])
    subprocess.check_call(['tar','xzf','lfw_aegan.tar.gz'])
    subprocess.check_call(['rm','lfw_aegan.tar.gz'])

In [8]:
data=numpy.load('tests/dmt2-lfw-multiple-attribute-test.npz')
pairs=list(data['pairs'][[0,1,2,4,5,6]]) # skip flushed face, not interesting
X=data['X']
X=X[:1]

In [9]:
K=100
delta = '0.4'
delta_params=[float(x.strip()) for x in delta.split(',')]
# t0=time.time()
result=[]
original=[]

In [10]:
for i in range(len(X)):
    result.append([])
    xX=X[i].decode('utf-8').replace('lfw','lfw_aegan')
    o=imageutils.read(xX)

In [11]:
image_dims=o.shape[:2]

In [12]:
if min(image_dims)<minimum_resolution:
    s=float(minimum_resolution)/min(image_dims)
    image_dims=(int(round(image_dims[0]*s)),int(round(image_dims[1]*s)))
    o=imageutils.resize(o,image_dims)

In [13]:
XF=model.mean_F([o])

/home/vk352/FaceDetection/deepfeatinterp/deepmodels_torch.py:137: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input_var = Variable(input, volatile=True).cuda()


In [14]:
original.append(o)

In [15]:
X = [x.decode('UTF-8') for x in X]

In [16]:
for j,(a,b) in enumerate([pairs[0]]):
    _,P,Q=make_manifolds(b.decode('UTF-8'),[a.decode('UTF-8')],[],X=X[0:1],N=1)
P=P[0]
Q=Q[0]
xP=[x.replace('lfw','lfw_aegan') for x in P]
xQ=[x.replace('lfw','lfw_aegan') for x in Q]

In [17]:
PF=model.mean_F(utils.image_feed(xP[:K],image_dims))
QF=model.mean_F(utils.image_feed(xQ[:K],image_dims))

In [18]:
if scaling=='beta':
    WF=(QF-PF)/((QF-PF)**2).mean()
elif scaling=='none':
    WF=(QF-PF)
max_iter=itera
init=o

In [19]:
for delta in delta_params:
    print(xX,b,delta)
    t2=time.time()
    Y=model.F_inverse(XF+WF*delta,max_iter=max_iter,initial_image=init)
    t3=time.time()
    print('{} minutes to reconstruct'.format((t3-t2)/60.0))
    result[-1].append(Y)
    max_iter=itera//2
    init=Y

images/lfw_aegan/Melchor_Cob_Castro/Melchor_Cob_Castro_0001.jpg b'Senior' 0.4


/home/vk352/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


0.33898085355758667 minutes to reconstruct


In [20]:
result=numpy.asarray(result)
original=numpy.asarray(original)

In [21]:
if 'color' in postprocess:
    result=utils.color_match(numpy.expand_dims(original,1),result)

Computing color match (1, 1, 200, 200, 3) (1, 1, 200, 200, 3)


In [22]:
m=imageutils.montage(numpy.concatenate([numpy.expand_dims(original,1),result],axis=1))

In [23]:
imageutils.write('results/demo1vk2.png',m)